# Coffee CNN Fine-tuning Runner

This notebook executes the coffee level detection CNN fine-tuning script using command line execution and logs all output to `finetune_log.txt` for monitoring and analysis.

## Features
- Real-time command execution with output capture
- Comprehensive logging to text file with timestamps
- Progress monitoring during training
- Results parsing and display
- Configurable training parameters

## 1. Import Required Libraries

Import subprocess, os, datetime, and other necessary libraries for command execution and file handling.

In [ ]:
import subprocess
import os
import sys
import time
import json
from datetime import datetime
from pathlib import Path
import threading
import queue
from IPython.display import display, clear_output
import ipywidgets as widgets

## 2. Setup Logging Configuration

Configure logging to write all output to `finetune_log.txt` with timestamps and proper formatting.

In [ ]:
# Setup logging configuration
LOG_FILE = "finetune_log.txt"
SCRIPT_PATH = "coffee_level_detection/training/finetune_coffeeCNN.py"

def setup_logging():
    """Initialize the log file with session header."""
    timestamp = datetime.now().isoformat()
    header = f"""
{'='*80}
Coffee CNN Fine-tuning Session Started
Timestamp: {timestamp}
Log File: {LOG_FILE}
Script: {SCRIPT_PATH}
{'='*80}

"""
    with open(LOG_FILE, 'w', encoding='utf-8') as f:
        f.write(header)
    print(f"✅ Logging initialized: {LOG_FILE}")

def log_message(message, print_also=True):
    """Log a message to the file with timestamp."""
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    log_entry = f"[{timestamp}] {message}\n"
    
    with open(LOG_FILE, 'a', encoding='utf-8') as f:
        f.write(log_entry)
    
    if print_also:
        print(f"[{timestamp}] {message}")

# Initialize logging
setup_logging()

## 3. Define Command Execution Function

Create a function to execute the fine-tuning script with real-time output capture and logging to the text file.

In [ ]:
def execute_finetune_command(command_args, real_time_output=True):
    """
    Execute the fine-tuning command with real-time output capture and logging.
    
    Args:
        command_args: List of command arguments
        real_time_output: Whether to show real-time output in notebook
    
    Returns:
        tuple: (return_code, stdout, stderr)
    """
    log_message(f"🚀 Executing command: {' '.join(command_args)}")
    
    try:
        # Start the process
        process = subprocess.Popen(
            command_args,
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,  # Merge stderr with stdout
            text=True,
            bufsize=1,
            universal_newlines=True
        )
        
        output_lines = []
        
        # Read output line by line in real-time
        while True:
            output = process.stdout.readline()
            if output == '' and process.poll() is not None:
                break
            if output:
                line = output.strip()
                output_lines.append(line)
                
                # Log to file
                log_message(line, print_also=False)
                
                # Show in notebook if requested
                if real_time_output:
                    print(line)
        
        # Wait for process to complete
        return_code = process.poll()
        
        # Final status
        if return_code == 0:
            log_message("✅ Command completed successfully")
        else:
            log_message(f"❌ Command failed with return code: {return_code}")
        
        return return_code, '\n'.join(output_lines), ''
        
    except Exception as e:
        error_msg = f"❌ Error executing command: {str(e)}"
        log_message(error_msg)
        return 1, '', str(e)

def read_log_tail(num_lines=20):
    """Read the last N lines from the log file."""
    try:
        with open(LOG_FILE, 'r', encoding='utf-8') as f:
            lines = f.readlines()
            return ''.join(lines[-num_lines:])
    except FileNotFoundError:
        return "Log file not found."
    except Exception as e:
        return f"Error reading log: {e}"

print("✅ Command execution functions defined")

## 4. Configure Fine-tuning Parameters

Set up command line arguments for the fine-tuning script including learning rate, epochs, batch size, and file paths.

In [ ]:
# Fine-tuning configuration parameters
config = {
    # Model paths
    'pretrained': 'coffeeCNN_finetuned_20251002_195526.pth',  # Use latest pretrained model
    'output': f'coffeeCNN_finetuned_{datetime.now().strftime("%Y%m%d_%H%M%S")}.pth',
    
    # Data paths
    'manual_levels': 'manual_levels',
    'img_dir': 'processed_images',
    
    # Training parameters
    'lr': 1e-6,              # Very conservative learning rate
    'epochs': 10,            # Max epochs (early stopping will likely trigger first)
    'batch_size': 8,         # Small batch size for stable training
    'num_classes': 11,       # Coffee levels 0-10
    'early_stopping': 3,     # Early stopping patience
    'dropout': 0.3,          # Dropout rate for regularization
    
    # Data filtering
    'max_per_class': 100,    # Maximum samples per class
    'min_per_class': 5,      # Minimum samples per class
    
    # Options
    'freeze_layers': True,   # Freeze early layers (set to False to add --no-freeze)
    'dry_run': False         # Set to True to only show data statistics
}

def build_command(config):
    """Build the command line arguments from configuration."""
    cmd = ['python', '-m', SCRIPT_PATH.replace('/', '.').replace('.py', '')]
    
    # Add arguments
    cmd.extend(['--pretrained', config['pretrained']])
    cmd.extend(['--output', config['output']])
    cmd.extend(['--manual-levels', config['manual_levels']])
    cmd.extend(['--img-dir', config['img_dir']])
    cmd.extend(['--lr', str(config['lr'])])
    cmd.extend(['--epochs', str(config['epochs'])])
    cmd.extend(['--batch-size', str(config['batch_size'])])
    cmd.extend(['--num-classes', str(config['num_classes'])])
    cmd.extend(['--early-stopping', str(config['early_stopping'])])
    cmd.extend(['--dropout', str(config['dropout'])])
    cmd.extend(['--max-per-class', str(config['max_per_class'])])
    cmd.extend(['--min-per-class', str(config['min_per_class'])])
    
    # Optional flags
    if not config['freeze_layers']:
        cmd.append('--no-freeze')
    if config['dry_run']:
        cmd.append('--dry-run')
    
    return cmd

# Build the command
command_args = build_command(config)

print("🔧 Fine-tuning Configuration:")
print("=" * 40)
for key, value in config.items():
    print(f"  {key}: {value}")
print("\n📝 Generated Command:")
print(f"  {' '.join(command_args)}")

log_message(f"Configuration: {json.dumps(config, indent=2)}")
log_message(f"Command: {' '.join(command_args)}")

## 5. Execute Fine-tuning Command

Run the finetune_coffeeCNN.py script using subprocess with parameter passing and real-time output streaming.

In [ ]:
# Execute the fine-tuning command
print("🚀 Starting Coffee CNN Fine-tuning...")
print("=" * 50)
print("📝 Real-time output will be shown below and logged to finetune_log.txt")
print("⏱️ This process may take several minutes to complete...")
print()

start_time = time.time()

# Execute the command with real-time output
return_code, stdout, stderr = execute_finetune_command(command_args, real_time_output=True)

end_time = time.time()
execution_time = end_time - start_time

# Log execution summary
summary = f"""
{'='*50}
EXECUTION SUMMARY
{'='*50}
Return Code: {return_code}
Execution Time: {execution_time:.2f} seconds ({execution_time/60:.2f} minutes)
Status: {'SUCCESS' if return_code == 0 else 'FAILED'}
Output Model: {config['output']}
Log File: {LOG_FILE}
{'='*50}
"""

log_message(summary)
print(summary)

if return_code != 0:
    print("❌ Fine-tuning failed! Check the log file for details.")
    if stderr:
        print(f"Error: {stderr}")
else:
    print("✅ Fine-tuning completed successfully!")
    print(f"💾 Model saved as: {config['output']}")
    print(f"📈 History saved as: {config['output'].replace('.pth', '_history.json')}")
    print(f"📋 Full log saved in: {LOG_FILE}")

## 6. Monitor Training Progress

Implement real-time monitoring of the training process by reading and displaying log file contents during execution.

In [ ]:
# Monitor training progress by examining the log file
def monitor_training_progress():
    """Monitor and display training progress from the log file."""
    print("📊 Training Progress Monitor")
    print("=" * 40)
    
    # Read the full log file
    try:
        with open(LOG_FILE, 'r', encoding='utf-8') as f:
            log_content = f.read()
        
        # Extract key information
        lines = log_content.split('\n')
        
        # Find training epochs
        epoch_lines = [line for line in lines if 'Epoch ' in line and '/' in line]
        
        if epoch_lines:
            print("🔄 Training Epochs Found:")
            for epoch_line in epoch_lines[-5:]:  # Show last 5 epochs
                # Extract just the relevant part
                if '] ' in epoch_line:
                    content = epoch_line.split('] ', 1)[1]
                    print(f"  {content}")
        
        # Find accuracy/loss information
        acc_lines = [line for line in lines if ('Train Acc:' in line or 'Val Acc:' in line)]
        if acc_lines:
            print("\n📈 Latest Training Metrics:")
            for acc_line in acc_lines[-4:]:  # Show last 4 accuracy lines
                if '] ' in acc_line:
                    content = acc_line.split('] ', 1)[1]
                    print(f"  {content}")
        
        # Find best model saves
        save_lines = [line for line in lines if 'Saved new best model' in line]
        if save_lines:
            print(f"\n💾 Best Model Updates: {len(save_lines)} saves")
            if save_lines:
                latest_save = save_lines[-1]
                if '] ' in latest_save:
                    content = latest_save.split('] ', 1)[1]
                    print(f"  Latest: {content}")
        
        # Check for early stopping
        early_stop_lines = [line for line in lines if 'Early stopping triggered' in line]
        if early_stop_lines:
            print(f"\n🛑 Early Stopping: {early_stop_lines[-1].split('] ', 1)[1]}")
        
        # Check for completion
        completion_lines = [line for line in lines if 'Fine-tuning complete' in line]
        if completion_lines:
            print(f"\n🎉 Training Status: Complete")
        
        # Show final validation accuracy
        final_acc_lines = [line for line in lines if 'Best validation accuracy:' in line]
        if final_acc_lines:
            final_acc = final_acc_lines[-1]
            if '] ' in final_acc:
                content = final_acc.split('] ', 1)[1]
                print(f"📊 {content}")
        
    except FileNotFoundError:
        print("❌ Log file not found. Run the training first.")
    except Exception as e:
        print(f"❌ Error reading log: {e}")

# Monitor the current training progress
monitor_training_progress()

## 7. Parse and Display Results

Parse the final results from the log file and display training statistics, best validation accuracy, and saved model information.

In [ ]:
def parse_final_results():
    """Parse and display comprehensive results from the training session."""
    print("📊 Final Results Analysis")
    print("=" * 50)
    
    try:
        # Check if history file was created
        history_file = config['output'].replace('.pth', '_history.json')
        
        if os.path.exists(history_file):
            print(f"📈 Training History File: {history_file}")
            try:
                with open(history_file, 'r') as f:
                    history = json.load(f)
                
                print("\n🎯 Training Summary:")
                training_history = history.get('training_history', {})
                
                if 'best_val_acc' in training_history:
                    print(f"  Best Validation Accuracy: {training_history['best_val_acc']:.2f}%")
                if 'best_epoch' in training_history:
                    print(f"  Best Epoch: {training_history['best_epoch'] + 1}")
                if 'total_epochs' in training_history:
                    print(f"  Total Epochs Trained: {training_history['total_epochs']}")
                
                # Show final losses/accuracies
                if 'train_losses' in training_history:
                    final_loss = training_history['train_losses'][-1]
                    print(f"  Final Training Loss: {final_loss:.4f}")
                
                if 'val_accuracies' in training_history:
                    final_val_acc = training_history['val_accuracies'][-1]
                    print(f"  Final Validation Accuracy: {final_val_acc:.2f}%")
                
                print(f"\n📊 Data Statistics:")
                data_stats = history.get('data_stats', {})
                if data_stats:
                    total_samples = sum(data_stats.values())
                    print(f"  Total Training Samples: {total_samples}")
                    print(f"  Classes Used: {list(data_stats.keys())}")
                    print(f"  Samples per Class: {data_stats}")
                
            except Exception as e:
                print(f"  ❌ Error reading history file: {e}")
        else:
            print(f"⚠️ History file not found: {history_file}")
        
        # Check if model file was created
        if os.path.exists(config['output']):
            model_size = os.path.getsize(config['output']) / (1024 * 1024)  # MB
            print(f"\n💾 Model File: {config['output']}")
            print(f"  Size: {model_size:.2f} MB")
            print(f"  Created: {datetime.fromtimestamp(os.path.getctime(config['output']))}")
        else:
            print(f"\n❌ Model file not found: {config['output']}")
        
        # Show log file stats
        if os.path.exists(LOG_FILE):
            log_size = os.path.getsize(LOG_FILE) / 1024  # KB
            print(f"\n📋 Log File: {LOG_FILE}")
            print(f"  Size: {log_size:.2f} KB")
            
            # Count important events in log
            with open(LOG_FILE, 'r', encoding='utf-8') as f:
                log_content = f.read()
            
            epoch_count = log_content.count('Epoch ')
            save_count = log_content.count('Saved new best model')
            error_count = log_content.count('❌')
            warning_count = log_content.count('⚠️')
            
            print(f"  Epochs Logged: {epoch_count}")
            print(f"  Model Saves: {save_count}")
            if error_count > 0:
                print(f"  Errors: {error_count}")
            if warning_count > 0:
                print(f"  Warnings: {warning_count}")
        
        print("\n✅ Results analysis complete!")
        
    except Exception as e:
        print(f"❌ Error parsing results: {e}")

def show_log_tail(lines=30):
    """Show the last N lines of the log file."""
    print(f"\n📝 Last {lines} lines of log file:")
    print("-" * 60)
    tail_content = read_log_tail(lines)
    print(tail_content)
    print("-" * 60)

# Parse and display final results
parse_final_results()

# Show recent log entries
show_log_tail(20)

## Additional Utilities

Helper functions for working with the fine-tuning process and log files.

In [ ]:
# Additional utility functions

def view_full_log():
    """Display the complete log file content."""
    try:
        with open(LOG_FILE, 'r', encoding='utf-8') as f:
            content = f.read()
        print("📋 Complete Log File Content:")
        print("=" * 80)
        print(content)
        print("=" * 80)
    except FileNotFoundError:
        print("❌ Log file not found.")
    except Exception as e:
        print(f"❌ Error reading log file: {e}")

def run_dry_run():
    """Run the fine-tuning script in dry-run mode to check data."""
    print("🏃 Running dry-run to check data availability...")
    
    # Create dry-run config
    dry_config = config.copy()
    dry_config['dry_run'] = True
    dry_config['output'] = 'dry_run_test.pth'  # Won't be created
    
    dry_command = build_command(dry_config)
    
    # Execute dry run
    return_code, stdout, stderr = execute_finetune_command(dry_command, real_time_output=True)
    
    if return_code == 0:
        print("✅ Dry run completed successfully - data is ready for training!")
    else:
        print("❌ Dry run failed - check data availability")
    
    return return_code == 0

def check_model_files():
    """Check what model files are available in the current directory."""
    print("📁 Available Model Files:")
    print("-" * 40)
    
    model_files = []
    for file in os.listdir('.'):
        if file.endswith('.pth'):
            size_mb = os.path.getsize(file) / (1024 * 1024)
            mod_time = datetime.fromtimestamp(os.path.getmtime(file))
            model_files.append((file, size_mb, mod_time))
            print(f"  {file}")
            print(f"    Size: {size_mb:.2f} MB")
            print(f"    Modified: {mod_time}")
            print()
    
    if not model_files:
        print("  No .pth model files found in current directory")
    
    return model_files

def cleanup_logs():
    """Archive the current log file."""
    if os.path.exists(LOG_FILE):
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        archive_name = f"finetune_log_{timestamp}.txt"
        os.rename(LOG_FILE, archive_name)
        print(f"📦 Log file archived as: {archive_name}")
        return archive_name
    else:
        print("❌ No log file to archive")
        return None

print("🛠️ Utility functions loaded:")
print("  - view_full_log(): Display complete log file")
print("  - run_dry_run(): Test data availability without training")
print("  - check_model_files(): List available model files")
print("  - cleanup_logs(): Archive current log file")

# Quick status check
print(f"\n📊 Current Status:")
print(f"  Log file exists: {os.path.exists(LOG_FILE)}")
print(f"  Expected model: {config['output']}")
print(f"  Model exists: {os.path.exists(config['output'])}")
print(f"  Script path: {SCRIPT_PATH}")